In [1]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/home/mrzaizai2k/code_Bao/ViT5/examples


In [2]:
import sys
sys.path.append("")

In [3]:
# !wget 'https://github.com/ThanhChinhBK/vietnews/archive/master.zip'
# !unzip 'master.zip'

In [4]:
import glob
import pandas as pd
pd.set_option('display.max_columns', None)
import concurrent.futures
from datasets import *
import datasets
from sklearn.model_selection import train_test_split
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# from vncorenlp import VnCoreNLP
import nltk
from nltk.tokenize import sent_tokenize



/home/mrzaizai2k/code_Bao/ViT5/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
OUTPUT_DIR = 'my_fine_tuned_t5_small_model'

In [6]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /home/mrzaizai2k/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Processing data

In [7]:
def listPaths(path):
  pathfiles = list()
  for pathfile in glob.glob(path):
    pathfiles.append(pathfile)
  return pathfiles

train_paths = listPaths('data/vietnews-master/data/train_tokenized/*')
val_paths = listPaths('data/vietnews-master/data/val_tokenized/*')
test_paths = listPaths('data/vietnews-master/data/test_tokenized/*')

In [8]:
def read_content(pathfile):
    """
    Input: Path of txt file
    Output: A dictionary has keys 'original' and 'summary'
    """
    with open(pathfile) as f:
      rows  = f.readlines()
      original = ' '.join(''.join(rows[4:]).split('\n'))
      summary = ' '.join(rows[2].split('\n'))
            
    return {'file' : pathfile,
              'original': original, 
              'summary': summary}

In [9]:
read_content(train_paths[0])

{'file': 'data/vietnews-master/data/train_tokenized/006157.txt.seg',
 'original': 'Tập 4 Thần_tượng Âm_nhạc nhí - Vietnam_Idol_Kids 2017 lên sóng tối 2/6 với màn tranh tài của top 8 thí_sinh nữ để lựa_chọn ra 5 tấm vé vào tiếp vòng trong . Trong đó , 3 giám_khảo Isaac - Văn_Mai_Hương - Bích_Phương được quyền lựa_chọn 3 thí_sinh . 2 thí_sinh còn lại sẽ do các giám_khảo khách mời quyết_định . Nữ ca_sĩ Văn_Mai_Hương xúc_động : “ Cô có may_mắn năm nay ngồi ở vị_trí ban giám_khảo , may_mắn hơn là được gặp con . Mỗi khi gặp Hiền , cô tự thấy bản_thân cô rất kém , bởi có lúc cô không trân_trọng cũng như không tin vào bản_thân ... Cảm_ơn con , bởi đôi_khi có những cái cô không bằng con được , đó là sự lạc_quan . Và cô tin , còn rất nhiều người phải học đức_tính lạc_quan này của Hiền . Con hát rất là hay ” . Đồng_tình với ý_kiến của đồng_nghiệp , Bích_Phương cũng xúc_động chia_sẻ : “ Giọt nước_mắt dành cho con là sự khâm_phục chứ không phải là thương_cảm . Cô chưa bao_giờ thấy con buồn . Từ lúc

In [10]:
def get_dataframe(pathfiles):
    with concurrent.futures.ProcessPoolExecutor() as executor:
      data = executor.map(read_content, pathfiles)
    
    # Make blank dataframe
    data_df = list()
    for d in data:
      data_df.append(d)
    data_df = pd.DataFrame(data_df)
    data_df.dropna(inplace = True)
    data_df = data_df.sample(frac=1).reset_index(drop=True)

    return data_df

In [11]:
train_df = get_dataframe(train_paths[:100])

In [12]:
val_df = get_dataframe(val_paths[:100])
test_df = get_dataframe(test_paths[:100])

In [13]:
test_df.head()

,file,original,summary
0,data/vietnews-master/data/test_tokenized/01053...,Cựu thủ_tướng Thái_Lan Yingluck_Shinawatra chà...,Người đứng đầu Lục_quân Hoàng_gia Thái_Lan cho...
1,data/vietnews-master/data/test_tokenized/00538...,""" Bị bắt quả_tang rồi . Vô_cùng thất_vọng vì T...","Tổng_thống Mỹ Donald_Trump tuyên_bố "" vô_cùng ..."
2,data/vietnews-master/data/test_tokenized/01056...,Tổng_thống Philippines_Rodrigo_Duterte . Bộ Nộ...,Quyết_định mới của chính_quyền Duterte gây ra ...
3,data/vietnews-master/data/test_tokenized/00334...,"Ngày 10-3 , một vụ tai_nạn giao_thông giữa xe_...","Sau va_chạm , 3 mẹ con ngã xuống_đường lọt vào..."
4,data/vietnews-master/data/test_tokenized/01424...,"Sáng ngày 22/7 , ông Trần_Đình_Ơn , Chủ_tịch U...","Sáng 22/7 , UBND T X.Phú Mỹ phối_hợp cùng các ..."


In [14]:
# test_df.to_parquet("data/vietnews/test.parquet", index=False)

In [15]:
# train_df = pd.read_parquet("data/vietnews/train.parquet")
# val_df = pd.read_parquet("data/vietnews/val.parquet")
# test_df = pd.read_parquet("data/vietnews/test.parquet")
# test_df.head()

In [16]:
# import matplotlib.pyplot as plt

# train_df['word_count'] = train_df['summary'].apply(lambda x: len(x.split()))

# # Plot the histogram
# plt.figure(figsize=(10, 6))
# plt.hist(train_df['word_count'], bins=100, edgecolor='black')
# plt.title('Histogram of Number of Words in Sentences')
# plt.xlabel('Number of Words')
# plt.ylabel('Frequency')
# plt.show()

## **Warm-starting RoBERTaShared for BBC XSum**

***Note***: This notebook only uses a few training, validation, and test data samples for demonstration purposes. To fine-tune an encoder-decoder model on the full training data, the user should change the training and data preprocessing parameters accordingly as highlighted by the comments.


### **Data Preprocessing**


In [17]:
# pretrained_model = "google/mt5-small"
pretrained_model = "huggingface-course/mt5-finetuned-amazon-en-es"
# facebook/bart-large-cnn
# "vinai/phobert-base"
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)


In [18]:
train_data =  Dataset.from_pandas(train_df)
val_data =  Dataset.from_pandas(val_df)
test_data =  Dataset.from_pandas(test_df)

In [19]:
batch_size=16  # change to 16 for full training

max_input_length = 512
max_target_length = 30


def process_data_to_model_inputs(batch):
    model_inputs = tokenizer(
        batch["original"],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(
        batch["summary"], max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# only use 32 training examples for notebook - DELETE LINE FOR FULL TRAINING
# train_data = train_data.select(range(32))

train_data_batch = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["file","original", "summary"],
)
train_data_batch.set_format(
    type="torch", columns=["input_ids", "attention_mask", "labels"],
)


# only use 16 training examples for notebook - DELETE LINE FOR FULL TRAINING
# val_data = val_data.select(range(16))

val_data_batch = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["file", "original", "summary"],
)
val_data_batch.set_format(
    type="torch", columns=["input_ids", "attention_mask", "labels"],
)

Map: 100%|██████████| 100/100 [00:00<00:00, 1546.40 examples/s]


In [20]:
tokenizer.pad_token_id

0

In [21]:
train_df.iloc[0]

file        data/vietnews-master/data/train_tokenized/0785...
original    Ngày 17-10 , Công_an TP Mỹ_Tho , tỉnh Tiền_Gia...
summary     Ông Trần_Văn_Mỹ trên đường tới ngân_hàng thì b...
Name: 0, dtype: object

In [22]:
a = tokenizer(str(train_df["summary"].iloc[0]))
print(str(train_df["summary"].iloc[0]))
a

Ông Trần_Văn_Mỹ trên đường tới ngân_hàng thì bị Lộc và Tâm đi xe_máy bình_xịt hơi cay xịt nhiều lần vào mặt , khống_chế cướp bọc tiền gần nửa tỉ đồng . 


{'input_ids': [259, 3920, 370, 1830, 1313, 290, 561, 2296, 290, 511, 5441, 534, 1079, 355, 2241, 259, 270, 908, 594, 1534, 290, 334, 3109, 394, 1135, 330, 933, 458, 1514, 297, 259, 793, 366, 6030, 3767, 2562, 290, 48319, 276, 330, 1253, 290, 329, 26804, 382, 3456, 104506, 259, 329, 26804, 677, 1990, 259, 280, 1313, 259, 793, 268, 326, 1335, 270, 259, 261, 1312, 2160, 290, 697, 1302, 317, 11871, 325, 330, 4929, 259, 270, 5059, 259, 318, 1313, 259, 272, 1642, 262, 259, 270, 4994, 5013, 259, 260, 259, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [23]:
tokenizer.convert_ids_to_tokens(train_data_batch[0]["labels"])

['▁',
 'Ô',
 'ng',
 '▁Tr',
 'ần',
 '_',
 'V',
 'ăn',
 '_',
 'M',
 'ỹ',
 '▁tr',
 'ên',
 '▁đ',
 'ường',
 '▁',
 't',
 'ới',
 '▁ng',
 'ân',
 '_',
 'h',
 'àng',
 '▁th',
 'ì',
 '▁b',
 'ị',
 '▁L',
 'ộ',
 '</s>']

In [24]:
# train_data_batch[0]

### **Warm-starting the Encoder-Decoder Model**

In [25]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model)

### **Fine-Tuning Warm-Started Encoder-Decoder Models**

The `Seq2SeqTrainer` that can be found under [examples/seq2seq/seq2seq_trainer.py](https://github.com/huggingface/transformers/blob/master/examples/seq2seq/seq2seq_trainer.py) will be used to fine-tune a warm-started encoder-decoder model.

Let's download the `Seq2SeqTrainer` code and import the module along with `TrainingArguments`.

In [26]:
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments
from dataclasses import dataclass, field
from typing import Optional

2024-06-09 11:23:45.369522: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-09 11:23:45.400427: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-09 11:23:45.917887: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


We need to add some additional parameters to make `TrainingArguments` compatible with the `Seq2SeqTrainer`. Let's just copy the `dataclass` arguments as defined in [this file](https://github.com/patrickvonplaten/transformers/blob/make_seq2seq_trainer_self_contained/examples/seq2seq/finetune_trainer.py).

Also, we need to define a function to correctly compute the ROUGE score during validation. ROUGE is a much better metric to track during training than only language modeling loss.

In [27]:
# # load rouge for validation
# rouge = datasets.load_metric("rouge")

# def compute_metrics(pred):
#     labels_ids = pred.label_ids
#     pred_ids = pred.predictions

#     # all unnecessary tokens are removed
#     pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
#     labels_ids[labels_ids == -100] = tokenizer.pad_token_id
#     label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

#     rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

#     return {
#         "rouge2_precision": round(rouge_output.precision, 4),
#         "rouge2_recall": round(rouge_output.recall, 4),
#         "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
#     }

In [28]:
import numpy as np
import evaluate

rouge_score = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE scores
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract the median scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [29]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors='pt')

In [30]:
features = [train_data_batch[i] for i in range(2)]
data_collator(features)

/home/mrzaizai2k/code_Bao/ViT5/venv/lib/python3.10/site-packages/transformers/data/data_collator.py:646: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


{'input_ids': tensor([[  441, 28811,   911,  ...,   273,   290,     1],
        [ 2826,   290,   270,  ...,   280,  1409,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([[  259,  3920,   370,  1830,  1313,   290,   561,  2296,   290,   511,
          5441,   534,  1079,   355,  2241,   259,   270,   908,   594,  1534,
           290,   334,  3109,   394,  1135,   330,   933,   458,  1514,     1],
        [  364,  1079,   317,  5779,   300,  3452,   297,   259,   270,  2184,
           290,   297,  2241,   394,  3316,   259,  1226,   259,   280,  1849,
           290,   280,  3577,   623,  1262,   270,   290, 11095,  4619,     1]]), 'decoder_input_ids': tensor([[    0,   259,  3920,   370,  1830,  1313,   290,   561,  2296,   290,
           511,  5441,   534,  1079,   355,  2241,   259,   270,   908,   594,
          1534,   290,   334,  3109,   394,  1135,   330,   933,   458,  1514],
        [    0,   364,  1079,   3

Cool! Finally, we start training.

In [31]:
# set training arguments - these params are not really tuned, feel free to change
logging_steps = len(train_data_batch) // batch_size

training_args = Seq2SeqTrainingArguments(
    output_dir= OUTPUT_DIR,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    # evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    # do_train=True,
    # do_eval=True,
    logging_steps=logging_steps,  # set to 2000 for full training
    save_steps=5000,  # set to 500 for full training
    eval_steps=75,  # set to 7500 for full training
    warmup_steps=3000,  # set to 3000 for full training
    num_train_epochs=10, #uncomment for full training
    overwrite_output_dir=True,
    save_total_limit=3,
    fp16=True, 
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model,
    args=training_args,
    train_dataset=train_data_batch,
    eval_dataset=val_data_batch,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

Step,Training Loss


TrainOutput(global_step=70, training_loss=0.0, metrics={'train_runtime': 25.0352, 'train_samples_per_second': 39.944, 'train_steps_per_second': 2.796, 'total_flos': 528749690880000.0, 'train_loss': 0.0, 'epoch': 10.0})

In [32]:
trainer.save_model("my_fine_tuned_t5_small_model")

events.out.tfevents.1717906393.DESKTOP-H2CRQMR.576476.0:   0%|          | 0.00/5.82k [00:00<?, ?B/s]


events.out.tfevents.1717906393.DESKTOP-H2CRQMR.576476.0: 100%|██████████| 5.82k/5.82k [00:00<00:00, 11.4kB/s]


events.out.tfevents.1717906393.DESKTOP-H2CRQMR.576476.0: 100%|██████████| 5.82k/5.82k [00:00<00:00, 6.09kB/s]
events.out.tfevents.1717907029.DESKTOP-H2CRQMR.578878.0: 100%|██████████| 5.53k/5.53k [00:00<00:00, 5.74kB/s]
training_args.bin: 100%|██████████| 5.30k/5.30k [00:00<00:00, 5.35kB/s]
Upload 3 LFS files: 100%|██████████| 3/3 [00:01<00:00,  2.02it/s]


In [33]:
# !gsutil -m cp -r '/content/training/*' 'gs://kaggle-vbdi-test/training_Data'

### **Evaluation**

Awesome, we finished training our dummy model. Let's now evaluated the model on the test data. We make use of the dataset's handy `.map()` function to generate a summary of each sample of the test data.

In [34]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model, use_fast=False)

# model = EncoderDecoderModel.from_pretrained(OUTPUT_DIR + "/checkpoint-4000")
model = AutoModelForSeq2SeqLM.from_pretrained("my_fine_tuned_t5_small_model")

model.to("cuda")

# test_data = datasets.load_dataset("xsum", split="test")

batch_size = 16  # change to 64 for full evaluation

# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    inputs = tokenizer(batch["original"], padding="max_length", truncation=True, max_length=256, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = model.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch

results = test_data.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["original"])

pred_str = results["pred"]
label_str = results["summary"]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Map:   0%|          | 0/100 [00:00<?, ? examples/s]/home/mrzaizai2k/code_Bao/ViT5/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Map: 100%|██████████| 100/100 [00:02<00:00, 42.58 examples/s]


In [35]:
rouge_output = rouge_score.compute(predictions=pred_str, references=label_str, use_stemmer=True)
rouge_output

{'rouge1': 0.2017155422679544,
 'rouge2': 0.06879877766768558,
 'rougeL': 0.16235664932212301,
 'rougeLsum': 0.1619359614723586}

In [36]:
for key,value in rouge_output.items():
  print(key)
  print(value)

rouge1
0.2017155422679544
rouge2
0.06879877766768558
rougeL
0.16235664932212301
rougeLsum
0.1619359614723586


In [37]:
i = 4
print('Prediction: ',pred_str[i])
print('Truth: ',label_str[i])
print('Content: ',test_data[i]['original'])

Prediction:  , tờ bản_đồ số 10
Truth:  Sáng 22/7 , UBND T X.Phú Mỹ phối_hợp cùng các cơ_quan_chức_năng liên_quan trên địa_bàn tỉnh Bà_Rịa - Vũng_Tàu ra_quân thực_hiện cưỡng_chế dự_án Alibaba_Tân_Thành_Center_City 1 do công_ty Địa_ốc Alibaba phân_phối đất nền . 
Content:  Sáng ngày 22/7 , ông Trần_Đình_Ơn , Chủ_tịch UBND xã Châu_Pha , T X.Phú Mỹ , tỉnh Bà_Rịa - Vũng_Tàu thông_qua Quyết_định cưỡng_chế đối_với ông Nguyễn_Ngọc_Sự ( ngụ TP. Hà_Nội ) . Địa_điểm cưỡng_chế gồm khu đất thuộc các thửa đất số 10 , 11 , 12 , 15 , 17 , 22 , 23 , 31 , 32 , 33 , 43 , 165 , 166 , 173 , 175 , tờ bản_đồ số 19 và thửa đất số 294 , tờ bản_đồ số 10 toạ_lạc tại thôn Tân_Tiến , xã Châu_Pha Có_mặt tại hiện_trường , ông Nguyễn_Văn_Thắm , Chủ_tịch UBND T X.Phú Mỹ cho biết : " Công_tác cưỡng_chế dự_kiến thực_hiện khoảng 2-3 ngày , cho đến khi thực_hiện xong . Việc cưỡng_chế được thực_hiện theo đúng quy_định pháp_luật " . " Trước đó , cơ_quan_chức_năng đã nhiều lần cho tự tháo_dỡ nhưng ông Sự không thực_hiện đúng

In [38]:
test_data[0]['file']

'data/vietnews-master/data/test_tokenized/010536.txt.seg'